In [1]:
import json
import os
import pandas as pd
import pymysql

In [5]:
def read_atributes(file):

    atributos = {}
#     remove = ['nFLows']

    with open(file, 'r') as signature_file:
        signature = json.load(signature_file)
        cardinalidade = signature['metadata']['cardinality']
        for step in signature['signature']:
            for atribute in signature['signature'][step]:
#                 if atribute not in remove:
                atributos.setdefault(atribute, [signature['signature'][step][atribute]['value_min'],signature['signature'][step][atribute]['value_max']])

    return atributos, cardinalidade

In [77]:
sig_path = '/home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/'

In [78]:
files = [os.path.join(sig_path, f) for f in os.listdir(sig_path) if os.path.isfile(os.path.join(sig_path, f))]

In [79]:
def query(atributos, cardinalidade, tabela):
    contar = ['nPortSrc', 'nPortDst', 'nAddrSrc', 'nAddrDst', 'nProto', 'nFLows']
    remove = ['nFLows']
    flags = {'all_have_tcp_A': 16, 
             'all_have_tcp_R': 4, 
             'all_have_tcp_S': 2, 
             'all_have_tcp_F': 1
            }
    dic = {'srcport': 'L4_SRC_PORT',
            'nPortSrc': 'L4_SRC_PORT',
            'nSrcPort': 'L4_SRC_PORT',
            'destport': 'L4_DST_PORT',
            'nPortDst': 'L4_DST_PORT',
            'nDestPort': 'L4_DST_PORT',
            'nAddrSrc': 'IPV4_SRC_ADDR',
            'nAddrDst': 'IPV4_DST_ADDR',
            'proto': 'PROTOCOL',
            'nProto': 'PROTOCOL',
            'nBytesSrc': 'IN_BYTES',
            'nBytesDst': 'OUT_BYTES',
            'nPcktsSrc': 'IN_PKTS',
            'nPktsSrc': 'IN_PKTS',
            'nPcktsDst': 'OUT_PKTS',
            'nPktsDst': 'OUT_PKTS'
            }
    
#     for atrib_name, atrib_value in atributos.items():
#         if atrib_name in remove:
#             prefixo = 'select count(*) as nFLows'
#         else:
#             prefixo = 'select count(*)'
    prefixo = 'select count(*) as contagem'
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in contar and atrib_name not in remove:
            prefixo += ', count(distinct ' + dic[atrib_name] + ') as ' + str(atrib_name)
        
    sql = prefixo + ' from ' + tabela + ' where '
    statement = ''
    cont = 0
    queries = []
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name not in contar and atrib_name not in flags:
            if cont != 0:
                statement += ' AND '
            statement += '(' + dic[atrib_name] + ' between ' + str(atrib_value[0]) + ' AND ' + str(atrib_value[1]) + ')'
            
        cont = cont + 1
    
    soma_flags = 0
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in flags:
            soma_flags += flags[atrib_name]
    
    # Se soma_flags for diferente de zero
    if soma_flags:
        statement += ' AND ((TCP_FLAGS = ' + str(soma_flags) + ') OR (TCP_FLAGS = ' + str(soma_flags+8) + ') OR (TCP_FLAGS = ' + str(soma_flags+32) + ') OR (TCP_FLAGS = ' + str(soma_flags+8+32) + '))'
            
    minimo = 1
    for interval in range(1000,51000,1000):
        if cardinalidade == '1-N':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR;'
        elif cardinalidade == 'N-1':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_DST_ADDR;'
        else:
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR, IPV4_DST_ADDR;'
        minimo = interval+1
        queries.append(sql + statement + sufixo)
#         print(sql + statement + sufixo)

    return queries

In [80]:
def consultar(query):
    db = pymysql.connect("200.145.216.190","root","1nstalation","ntopng")
    cursor = db.cursor()
    try:
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()
        db.close()
        return columns, results
    except:
        print ("Error: unable to fecth data")
        return [], ()
        db.close()

In [81]:
queries = []
vp = 0
vn = 0
fp = 0
fn = 0

total = 50 * len(files) * 2

for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos, cardinalidade = read_atributes(file)
    queries = query(atributos, cardinalidade, 'scanvuln_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro positivo
        for coluna in colunas:
            for valor in df[coluna].values:
                if coluna == 'contagem':
                    if valor == 0:
                        # É um falso negativo
                        classificador = 0
                elif not(atributos[coluna][0] <= valor <= atributos[coluna][1]):
                    # É um falso negativo
                    classificador = 0
        
        if (classificador):
            vp = vp + 1
        else:
            fn = fn + 1
    
    queries = query(atributos, cardinalidade, 'leg_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro negativo
        for coluna in colunas:
            for valor in df[coluna].values:
                if coluna == 'contagem':
                    if valor > 0:
                        # É um falso positivo
                        classificador = 0
                elif atributos[coluna][0] <= valor <= atributos[coluna][1]:
                    # É um falso positivo
                    classificador = 0
                    
        if (classificador):
            vn = vn + 1
        else:
            fp = fp + 1
        
    
print('\nVerdadeiro Positivo: ' + str(vp))
print('\nFalso Negativo: ' + str(fn))
print('\nFalso Positivo: ' + str(fp))
print('\nVerdadeiro Negativo: ' + str(vn))

print('\n\nSensibilidade = ' + str(vp/(vp + fn)))
print('\nEspecificidade = ' + str(vn/(vn + fp)))
print('\nAcurácia = ' + str((vp + vn)/total))
print('\nPrecisão = ' + str(vp/(vp + fp)))



Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln10.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln7.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln8.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln2.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln6.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln9.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln1.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln5.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-master/Assinaturas_sanitizadas/scanvuln/scanvuln3.sig


Assinatura: /home/amanda/Documents/TCC/TCC-tests-mas

In [45]:
colunas, resultados = consultar('select count(*) as contagem, count(distinct L4_SRC_PORT) as nPortSrc from leg_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND (idx between 40001 AND 41000) group by IPV4_SRC_ADDR, IPV4_DST_ADDR;')

In [46]:
colunas

['contagem', 'nPortSrc']

In [47]:
resultados

((1, 1), (1, 1))

In [48]:
df = pd.DataFrame(list(resultados), columns=colunas)

In [49]:
df

,contagem,nPortSrc
0,1,1
1,1,1


In [52]:
classificador = 1 # É um verdadeiro negativo
for coluna in colunas:
    for valor in df[coluna].values:
        if coluna == 'contagem':
            if valor > 0:
                # É um falso positivo
                classificador = 0
        elif atributos[coluna][0] <= valor <= atributos[coluna][1]:
            # É um falso positivo
            classificador = 0

KeyError: 'nPortSrc'

In [51]:
classificador

0